In [1]:
import os
os.chdir("../")
os.chdir("FinalDataset")

In [2]:
os.getcwd()

'/Volumes/Rohith/url_classification_dl/FinalDataset'

In [3]:
import pandas as pd
import tensorflow as ts
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
from keras import backend as K

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [4]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [5]:
data = pd.read_csv("feature.csv")

In [6]:
data.drop(columns='Unnamed: 0',inplace=True)
data.replace(True,1,inplace = True)
data.replace(False,0,inplace = True)
y = data["File"]

In [7]:
data = data.drop(columns = "File")

In [8]:
encoder = LabelEncoder()
encoder.fit(y)
Y = encoder.transform(y)

In [9]:
scaler = MinMaxScaler(feature_range=(0, 2))
X = scaler.fit_transform(data)
X = pd.DataFrame(X)

In [10]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

input_dim = len(data.columns)
model = Sequential()
model.add(Dense(256, input_dim = input_dim , activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy',f1_m,precision_m, recall_m] )
    

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [12]:
model.fit(X_train,np_utils.to_categorical(y_train),epochs = 50,validation_split=0.3, batch_size = 128)

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpvs1_9qaq.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpvs1_9qaq.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
224/224 [==============================] - 11s 5ms/step - loss: 1.3070 - accuracy: 0.4209 - f1_m: 0.2093 - precision_m: 0.5108 - recall_m: 0.1413 - val_loss: 0.9785 - val_accuracy: 0.5956 - val_f1_m: 0.5786 - val_precision_m: 0.7300 - val_recall_m: 0.4798
Epoch 2/50
224/224 [==============================] - 0s 2ms/step - loss: 0.9297 - accuracy: 0.6278 - f1_m: 0.6028 - precision_m:

224/224 [==============================] - 0s 2ms/step - loss: 0.4464 - accuracy: 0.8237 - f1_m: 0.8244 - precision_m: 0.8779 - recall_m: 0.7776 - val_loss: 0.4879 - val_accuracy: 0.8222 - val_f1_m: 0.8171 - val_precision_m: 0.8697 - val_recall_m: 0.7709
Epoch 29/50
224/224 [==============================] - 0s 2ms/step - loss: 0.4493 - accuracy: 0.8272 - f1_m: 0.8246 - precision_m: 0.8767 - recall_m: 0.7787 - val_loss: 0.4806 - val_accuracy: 0.8194 - val_f1_m: 0.8140 - val_precision_m: 0.8688 - val_recall_m: 0.7660
Epoch 30/50
224/224 [==============================] - 0s 2ms/step - loss: 0.4303 - accuracy: 0.8353 - f1_m: 0.8320 - precision_m: 0.8833 - recall_m: 0.7866 - val_loss: 0.4672 - val_accuracy: 0.8281 - val_f1_m: 0.8218 - val_precision_m: 0.8739 - val_recall_m: 0.7758
Epoch 31/50
224/224 [==============================] - 0s 2ms/step - loss: 0.4379 - accuracy: 0.8302 - f1_m: 0.8290 - precision_m: 0.8796 - recall_m: 0.7843 - val_loss: 0.4747 - val_accuracy: 0.8163 - val_f1_m: 

In [13]:
y_pred = model.predict(X_test)

In [14]:
predicted = np.argmax(y_pred, axis=1)

In [15]:
from sklearn.metrics import classification_report
print(accuracy_score(y_test,predicted))

0.8448617325606983


In [16]:
target_names = ['Benign','Defacement','Malware','Phishing','Spam']
print(classification_report(y_test, predicted, target_names=target_names))

                            precision    recall  f1-score   support

     Benign_list_big_final       0.95      0.93      0.94      2735
DefacementSitesURLFiltered       0.78      0.73      0.75      2499
           Malware_dataset       0.86      0.89      0.88      2834
          phishing_dataset       0.77      0.75      0.76      2477
              spam_dataset       0.84      0.90      0.87      3088

                  accuracy                           0.84     13633
                 macro avg       0.84      0.84      0.84     13633
              weighted avg       0.84      0.84      0.84     13633



In [ ]:
os.chdir("../")
os.chdir("models")
model.save("Model_v1")